In [3]:
import os
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
from datatable import dt, fread
# With python v>3.9 need to install datatable from source for the time being
# pip install git+https://github.com/h2oai/datatable

In [4]:
annot = pd.read_csv("PRJNA383416_extracted_condition_genotype_selected.csv")
annot.head()

,run_accession,sample_accession,study_accession,tax_id,scientific_name,instrument_model,library_strategy,read_count,run_alias,sample_alias,fastq_ftp,fastq_md5,condition,genotype
0,SRR5909693,SAMN06764809,PRJNA383416,4577,Zea mays,NextSeq 500,RNA-Seq,1750117,5949_8601_26615_HK537BGXX_KAKRNA6_10346261_GRo...,Maize 3 prime RNASeq,ftp.sra.ebi.ac.uk/vol1/fastq/SRR590/003/SRR590...,817569d2f8d6c2f2bd771e1da0b39a98,GRoot,4226
1,SRR5909692,SAMN06764809,PRJNA383416,4577,Zea mays,NextSeq 500,RNA-Seq,10898114,5918_8601_26597_HLN5JBGXX_KAKRNA5_GShoot_4226_...,Maize 3 prime RNASeq,ftp.sra.ebi.ac.uk/vol1/fastq/SRR590/002/SRR590...,ea1a59a8ec2883bbc77487b180804d6d,GShoot,4226
2,SRR5909701,SAMN06764809,PRJNA383416,4577,Zea mays,NextSeq 500,RNA-Seq,5272706,6125_8601_27527_HJ5W5BGXX_KAKRNA16_Kern_4226_T...,Maize 3 prime RNASeq,ftp.sra.ebi.ac.uk/vol1/fastq/SRR590/001/SRR590...,aabfb6561a197ebee20d2f67fda29726,Kern,4226
3,SRR5909694,SAMN06764809,PRJNA383416,4577,Zea mays,NextSeq 500,RNA-Seq,4329387,6122_8601_27530_HJ7FKBGXX_KAKRNA28_L3Base_4226...,Maize 3 prime RNASeq,ftp.sra.ebi.ac.uk/vol1/fastq/SRR590/004/SRR590...,25f75be94c90a7fd90f5d80d73e05bca,L3Base,4226
4,SRR5909695,SAMN06764809,PRJNA383416,4577,Zea mays,NextSeq 500,RNA-Seq,6719535,6122_8601_27530_HJ7FKBGXX_KAKRNA28_L3Mid_4226_...,Maize 3 prime RNASeq,ftp.sra.ebi.ac.uk/vol1/fastq/SRR590/005/SRR590...,97136121885635a71aa418b5963fb555,L3Mid,4226


In [5]:
try:
    os.mkdir("ramdisk/geno_kmers")
except:
    None

genotypes = set(annot["genotype"].values)
genotype_counter = len(genotypes)
print("Total genotypes:", genotype_counter)
for geno in genotypes:
    acc = annot[annot["genotype"] == geno]["run_accession"].values.tolist()
    if len(acc)==1:
        os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(acc[0]))
        os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(acc[0]))
        os.system('pigz --decompress ramdisk/*')
        os.system('mv ramdisk/{0}.kmc_suf ramdisk/geno_kmers/{1}.kmc_suf'.format(acc[0], geno))
        os.system('mv ramdisk/{0}.kmc_pre ramdisk/geno_kmers/{1}.kmc_pre'.format(acc[0], geno))
    if len(acc)>=2:
        fl1, fl2 = acc[0], acc[1]
        os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(fl1))
        os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(fl1))
        os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(fl2))
        os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(fl2))
        os.system('pigz --decompress ramdisk/*')
        os.system('kmc_tools simple ramdisk/{0} ramdisk/{1} union ramdisk/kmer_union0'.format(fl1, fl2))
        os.system('rm {0}/{1}.kmc_suf {0}/{1}.kmc_pre {0}/{2}.kmc_suf {0}/{2}.kmc_pre'.format("ramdisk", fl1, fl2))
    if len(acc)==2:
        os.system('mv ramdisk/kmer_union0.kmc_suf ramdisk/geno_kmers/{0}.kmc_suf'.format(geno))
        os.system('mv ramdisk/kmer_union0.kmc_pre ramdisk/geno_kmers/{0}.kmc_pre'.format(geno))
    if len(acc)>2:
        acc = acc[2:]
        # kmc_tools can't overwrite a file so have to iterate back and 
        # forth with the output file to union merge multiple kmer files
        # This also seems to be an easier solution than the internal 'complex'
        count = 1
        for fl in acc:
            os.system('cp kmc/{0}.kmc_suf.gz ramdisk/'.format(fl))
            os.system('cp kmc/{0}.kmc_pre.gz ramdisk/'.format(fl))
            os.system('pigz --decompress ramdisk/*')
            prv, nxt = str((count-1)%2), str(count%2)
            os.system('kmc_tools simple ramdisk/kmer_union{0} ramdisk/{1} union ramdisk/kmer_union{2}'.format(prv, fl, nxt))
            os.system('rm {0}/{1}.kmc_suf {0}/{1}.kmc_pre {0}/kmer_union{2}.kmc_suf {0}/kmer_union{2}.kmc_pre'.format("ramdisk", fl, prv))
            count+=1
        # Done union merge of group kmers. Read table 
        os.system('mv ramdisk/kmer_union{0}.kmc_suf ramdisk/geno_kmers/{1}.kmc_suf'.format(nxt, geno))
        os.system('mv ramdisk/kmer_union{0}.kmc_pre ramdisk/geno_kmers/{1}.kmc_pre'.format(nxt, geno))
    genotype_counter = genotype_counter - 1
    print("Genotypes remaining:", genotype_counter)


Total genotypes: 300


pigz: skipping: ramdisk/geno_kmers is a directory
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union0.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union0.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union1.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union1.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union0.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union0.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union1.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union1.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is

Genotypes remaining: 299


pigz: skipping: ramdisk/geno_kmers is a directory
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union0.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union0.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union1.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union1.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union0.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union0.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union1.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union1.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is

Genotypes remaining: 298


...

Genotypes remaining: 2


pigz: skipping: ramdisk/geno_kmers is a directory
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union0.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union0.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union1.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union1.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 2: 61% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union0.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union0.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union1.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union1.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_k

Genotypes remaining: 1


pigz: skipping: ramdisk/geno_kmers is a directory
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union0.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union0.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union1.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union1.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union0.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union0.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 
pigz: skipping: ramdisk/geno_kmers is a directory
pigz: skipping: ramdisk/kmer_union1.kmc_pre does not have compressed suffix
pigz: skipping: ramdisk/kmer_union1.kmc_suf does not have compressed suffix
in1: 100% in2: 100% 


Genotypes remaining: 0


# Get the combined kmer file for all genotypes 

In [6]:
genotypes = list(set(annot["genotype"].values))
genotype_counter = len(genotypes)
print("Total genotypes:", genotype_counter)

fl1, fl2 = genotypes[0], genotypes[1]
os.system('kmc_tools simple ramdisk/geno_kmers/{0} ramdisk/geno_kmers/{1} union ramdisk/kmer_union0'.format(fl1, fl2))

genotype_counter = genotype_counter - 2
print("Total genotypes:", genotype_counter)

count = 1
for geno in genotypes[2:]:
    # kmc_tools can't overwrite a file so have to iterate back and 
    # forth with the output file to union merge multiple kmer files
    # This also seems to be an easier solution than the internal 'complex'
    prv, nxt = str((count-1)%2), str(count%2)
    os.system('kmc_tools simple ramdisk/kmer_union{0} ramdisk/geno_kmers/{1} union ramdisk/kmer_union{2}'.format(prv, geno, nxt))
    os.system('rm {0}{1}.kmc_suf {0}{1}.kmc_pre'.format("ramdisk/kmer_union", prv))
    count+=1

    genotype_counter = genotype_counter - 1
    print("Genotypes remaining:", genotype_counter)

Total genotypes: 300


in1: 100% in2: 100% 


Total genotypes: 298


in1: 100% in2: 100% 


Genotypes remaining: 297


...

Genotypes remaining: 2


in1: 100% in2: 100% 


Genotypes remaining: 1


in1: 100% in2: 100% 


Genotypes remaining: 0


# Combine k-mers to Hapmap file